#PRAKTIKUM 8
# Evaluasi Beberapa Algoritma Klasifikasi

Tujuan :
Setelah mengikuti praktikum ini diharapkan praktikan dapat:
1.   Memahami evaluasi beberapa metrik untuk mengukur kinerja klasifikasi
2.   Mahasiswa dapat memahami confusion matrix dan peranannya untuk evaluasi klasifikasi
3. Memahami dan mampu menggunakan evaluasi K-fold








Nama  : Dani Adrian

NIM   : 225150201111009

Kelas : PPM-E

##Dasar Teori

Validasi klasifikasi

Metode klasifikasi dibangun untuk dapat belajar dan diharapkan dapat membuat prediksi atau keputusan suatu data baru berdasarkan data yang sudah ada. Proses pembelajaran terhadap dataset yang sudah ada untuk mendapatkan model. Selanjutnya suatu data baru yang belum teridentifikasi dapat ditentukan label atau kelasnya berdasarkan model yang telah dibangun.

Proses pembelajaran yang dibangun terhadap suatu dataset dilakukan dengan cara membagi suatu data menjadi dua bagian yakni sebagian digunakan sebagai data pelatihan dan sebagian lainnya sebagai data validasi. Besaran jumlah masing-masing data pelatihan dan data validasi dapat ditentukan (50%,50%) atau (75%,25%).

Tahap selanjutnya data pelatihan akan digunakan untuk membangun model dengan menentukan parameter terbaik dari masing-masing model berdasarkan dataset yang ada. Kinerja terbaik dari suatu model yang dikembangkan diukur menggunakan beberap tolok ukur sebagai berikut:
- presision
- recall
- specifity
- akurasi
Masing-masing metrik tersebut ditentukan berdasarkan confusion matrix.



Confusion matrix atau matriks kesalahan adalah suatu tabel yang merepresentasikan kinerja suatu algoritma. Setiap baris matriks mewakili instance di kelas yang diprediksi sementara setiap kolom mewakili instance di kelas yang sebenarnya (atau sebaliknya).

#Beberapa Tahapan Dalam Valisasi Klasifikasi


Beberapa metode klasifikasi yang telah dipelajari mempunyai kelebihan dan kekurangannya. Perbandingan kinerja antar metode klasifikasi ini diukur menggunakan beberapa metrik. Pada praktikum ini akan dibahas beberapa metrik untuk mengukur kinerja klasifikasi.
Tahapan untuk mengevaluasi suatu metode klasifikasi dilakukan sebagai berikut:
- Pisahkan set data validasi.
- Siapkan test harness untuk menggunakan validasi silang 10 kali lipat.
- Buat beberapa model berbeda untuk memprediksi variety dari pengukuran bunga.
- Pilih model terbaik.

##1. Tentukan Metode Klasifikasi yang Akan Dievaluasi

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

In [5]:
# evaluate each model in turn
results = []
names = []

##2. Baca Dataset
Pada contoh ini terdapat tiga buah dataset yakni :
- dataR2
- iris data
- breast cancer

Kita akan gunakan dataR2 sebagai contoh sedangkan dua data berikutnya akan digunakan sabagai latihan.

In [6]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv

--2024-05-23 06:02:10--  https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘dataR2.csv’

dataR2.csv              [ <=>                ]   7.49K  --.-KB/s    in 0s      

2024-05-23 06:02:10 (75.5 MB/s) - ‘dataR2.csv’ saved [7665]



In [7]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-05-23 06:02:10--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-05-23 06:02:10 (16.8 MB/s) - ‘iris.csv’ saved [3975/3975]



In [8]:
! wget https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv

--2024-05-23 06:02:10--  https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3664 (3.6K) [text/plain]
Saving to: ‘breast_cancer.csv’

breast_cancer.csv   100%[===================>]   3.58K  --.-KB/s    in 0s      

2024-05-23 06:02:10 (50.6 MB/s) - ‘breast_cancer.csv’ saved [3664/3664]



##3. Tampilkan Dataset

In [9]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('dataR2.csv')
dataset

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1
...,...,...,...,...,...,...,...,...,...,...
111,45,26.850000,92,3.330,0.755688,54.6800,12.100000,10.96000,268.230,2
112,62,26.840000,100,4.530,1.117400,12.4500,21.420000,7.32000,330.160,2
113,65,32.050000,97,5.730,1.370998,61.4800,22.540000,10.33000,314.050,2
114,72,25.590000,82,2.820,0.570392,24.9600,33.750000,3.27000,392.460,2


###3.1. Membuat Set Data Validasi
Tujuan evaluasi adalah memperkirakan keakuratan model terbaik pada data tak terlihat dengan mengevaluasinya pada data tak terlihat aktual. Evaluasi ini dilakukan dengan cara membagi dataset menjadi dua bagian yakni sebagian digunakan sebagai data latih dan sebagian yang lain digunakan sebagai data uji. Beberapa praktikum sebelumnya telah dipraktekkan dengan menggunakan metode train_test_split:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)


Sebelum membuat data latih dan data uji. Langkah pertama pisahkan antara data atribut dan data label suatu dataset. Karena dataset data2R terdiri dari 9 atribut dan 1 label.  Maka kita catat X sebagai data dan y sebagai label.

In [10]:
from sklearn.model_selection import train_test_split
array = dataset.values
X = array[:,0:9]
y = array[:,9]
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

###3.2 K-Fold Cross Validation
K-Fold Cross Validation adalah prosedur pengambilan sampel ulang yang digunakan untuk mengevaluasi model pembelajaran mesin pada sampel data terbatas. Prosedur ini memiliki parameter tunggal yang disebut K yang mengacu pada jumlah grup yang akan dipecah menjadi sampel data tertentu. Karena itu, prosedur ini sering disebut K-fold cross validation. Ketika nilai spesifik untuk K dipilih, nilai tersebut dapat digunakan sebagai pengganti K dalam referensi ke model, seperti K = 10 menjadi validasi silang 10 kali lipat.

Validasi silang terutama digunakan dalam pembelajaran mesin terapan untuk memperkirakan kemampuan model pembelajaran mesin pada data yang tidak terlihat. Yaitu, menggunakan sampel terbatas untuk memperkirakan bagaimana performa model secara umum saat digunakan untuk membuat prediksi pada data yang tidak digunakan selama pelatihan model.

Prosedur umumnya adalah sebagai berikut:
- Acak kumpulan data berdasarkan id
- Pisahkan kumpulan data menjadi kelompok K
- Untuk setiap grup unik:
  - Ambil kelompok sebagai kelompok data uji
  - Ambil kelompok yang tersisa sebagai kumpulan data pelatihan
  - Catat hasil pengujian masing-masing grup

K-fold ini digunakan untuk validasi model yang digunakan. Evaluasi dilakukan pada dataset latih. Pada praktikum ini digunakan k=10, berarti akan membagi dataset latih menjadi 10 bagian. Satu bagian digunakan untuk testing dan sisanya sebagai data latih.

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
for name, model in models:
  kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=
'accuracy')
results.append(cv_results)
names.append(name)
print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

SVM: 0.465556 (0.047778)


##4. Buat Prediksi

Kita harus memilih algoritma yang akan digunakan untuk membuat prediksi.

Kita dapat menyesuaikan model pada seluruh dataset pelatihan dan membuat prediksi pada dataset validasi.

In [16]:
model = SVC(gamma='auto')
model.fit(X_train, y_train)
predictions = model.predict(X_validation)

##5. Evaluasi Prediksi
Kita dapat mengevaluasi prediksi dengan membandingkannya dengan hasil yang diharapkan dalam set validasi, lalu menghitung akurasi klasifikasi, serta confusion matrix dan laporan klasifikasi.

In [17]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_validation, predictions))

              precision    recall  f1-score   support

         1.0       0.25      1.00      0.40         6
         2.0       0.00      0.00      0.00        18

    accuracy                           0.25        24
   macro avg       0.12      0.50      0.20        24
weighted avg       0.06      0.25      0.10        24



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Tugas :

1. Lakukan perbandingan klasifikasi antara SVM, Naive Bayes, dan KNN untuk dataset iris dan breast cancer
2. Pada kedua dataset lakukan proses normalisasi data dan preproses data untuk menangani data yang hilang jika ada.

**Note: Tambahkan blok kode apabila diperlukan**

*Metode Normalisasi*

In [20]:
def minmax(df_input):
  list_fitur = df_input.columns[:-1]
  for fitur in list_fitur:
    max = df_input[fitur].max()
    min = df_input[fitur].min()
    df_input[fitur] = (df_input[fitur]-min)/(max-min)
  return df_input

*Metode Cetak Rentang*

In [22]:
def cetak_rentang(df_input):
  list_fitur = df_input.columns[:-1]
  for fitur in list_fitur:
    max = df_input[fitur].max()
    min = df_input[fitur].min()
    print("Rentang fitur ",fitur," adalah ",max-min)

*Metode Imputasi*

In [23]:
def imputasi(df_input):
    list_columns = df_input.columns
    class_column = list_columns[-1]
    for column in list_columns[:-1]:
        df_input[column].fillna(df_input.groupby(class_column)[column].transform('mean'), inplace=True)

    return df_input

###**Dataset Iris**

In [24]:
dataset_iris = pd.read_csv('iris.csv')

In [25]:
cetak_rentang(dataset_iris)

Rentang fitur  sepal.length  adalah  3.6000000000000005
Rentang fitur  sepal.width  adalah  2.4000000000000004
Rentang fitur  petal.length  adalah  5.9
Rentang fitur  petal.width  adalah  2.4


In [26]:
data_normal=minmax(dataset_iris)

In [30]:
iris = data_normal.values
Xi = iris[:,:-1]
yi = iris[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(Xi, yi, test_size=0.2, random_state=42)

In [31]:
print(X_train.shape[0])
print(y_test.shape[0])

120
30


In [32]:
# Prediksi KNN
from sklearn.metrics import accuracy_score
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_preds = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_preds)
print(knn_accuracy)

1.0


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [33]:
# Prediksi Naive Bayes (NB)
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_preds = nb_model.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_preds)
print(nb_accuracy)

1.0


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
# Prediksi SVM
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_preds)
print(svm_accuracy)

1.0


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


###**Dataset Breast Cancer**

In [40]:
dataset_bc = pd.read_csv('breast_cancer.csv')

In [41]:
cetak_rentang(dataset_bc)

Rentang fitur  id  adalah  170394
Rentang fitur  clump_thickness  adalah  9.0
Rentang fitur  cell_size  adalah  9.0
Rentang fitur  cell_shape  adalah  9.0
Rentang fitur  marginal_adhesion  adalah  9.0
Rentang fitur  epithelial_cell_size  adalah  9.0
Rentang fitur  bare_nucklei  adalah  9.0
Rentang fitur  bland_chromatin  adalah  8.0
Rentang fitur  normal_nucleoli  adalah  9.0
Rentang fitur  mitoses  adalah  9.0


In [44]:
data_normal_bc=minmax(dataset_bc)

In [46]:
data_normal_bc.isnull().sum()

id                       0
clump_thickness         10
cell_size               10
cell_shape              10
marginal_adhesion       10
epithelial_cell_size    10
bare_nucklei            10
bland_chromatin         10
normal_nucleoli         10
mitoses                 10
diagnosis                0
dtype: int64

In [47]:
data_imputasi_bc= imputasi(data_normal_bc)

In [48]:
data_imputasi_bc.isnull().sum()

id                      0
clump_thickness         0
cell_size               0
cell_shape              0
marginal_adhesion       0
epithelial_cell_size    0
bare_nucklei            0
bland_chromatin         0
normal_nucleoli         0
mitoses                 0
diagnosis               0
dtype: int64

In [49]:
cancer = data_imputasi_bc.values
Xc = cancer[:,:-1]
yc = cancer[:,-1:]
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(Xc, yc, test_size=0.2, random_state=42)

In [50]:
# Prediksi KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_c, y_train_c)
knn_preds = knn_model.predict(X_test_c)
knn_accuracy = accuracy_score(y_test_c, knn_preds)
print(knn_accuracy)

0.95


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [51]:
# Prediksi Naive Bayes (NB)
nb_model = GaussianNB()
nb_model.fit(X_train_c, y_train_c)
nb_preds = nb_model.predict(X_test_c)
nb_accuracy = accuracy_score(y_test_c, nb_preds)
print(nb_accuracy)

0.95


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
# Prediksi SVM
svm_model = SVC()
svm_model.fit(X_train_c, y_train_c)
svm_preds = svm_model.predict(X_test_c)
svm_accuracy = accuracy_score(y_test_c, svm_preds)
print(svm_accuracy)

1.0


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Penjelasan

1.   Jelaskan hasil prediksi dari dataset iris pada setiap model yang digunakan! Model apa yang memiliki tingkat akurasi tertinggi?
2.   Jelaskan hasil prediksi dari dataset breast cancer pada setiap model yang digunakan! Model apa yang memiliki tingkat akurasi tertinggi?

Jawaban:

1.   Hasil prediksi dataset iris Knn: 1.0 Nb: 1.0 SVM: 1.0. Ketiga model cocok dengan dataset iris

2. Hasil prediksi dataset breast cancer Knn: 0,95 Nb: 0,95 SVM: 1,0. Model SVM yang memiliki tingkat akurasi yang tinggi. yang berarti model svm cocok dengan dataset breast cancer

